In [1]:
#Import needed libraries 

In [2]:
import requests
import json
import pandas as pd
from getpass import getpass

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
#Get token from .env doc

In [5]:
token = os.getenv("token")

In [6]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [7]:
#Connect to MongoDB

In [8]:
import pymongo
client = pymongo.MongoClient("localhost:27017")
db = client["London"]
c = db.get_collection("companies")

In [9]:
list_ = list(c.find(projection= { "_id":0, "offices.city":1}))
city_count = []
for company in list_:
    for office in company["offices"]:
        city_count.append(office["city"])     

count = pd.Series(city_count).value_counts()
print("City Count")
print(count.head())

City Count
San Francisco    906
New York         837
                 646
London           616
Seattle          221
Name: count, dtype: int64


In [10]:
#Checking existing categories --> Using desing and games_video
list_ = list(c.find(projection= { "_id":0, "category_code":1}))
category_count = []
for companies in list_:
    category_count.append(companies["category_code"])     

count = pd.Series(category_count).value_counts()

In [11]:
#Searching for the location of the design companies
condition_1 = {"category_code": "design"}
projection = {"_id":0, "name":1, "category_code":1, "offices.city":1}
list(c.find(condition_1, projection))

[{'name': '99designs',
  'category_code': 'design',
  'offices': [{'city': 'San Francisco'},
   {'city': 'Collingwood'},
   {'city': 'Berlin'}]},
 {'name': 'Graticle',
  'category_code': 'design',
  'offices': [{'city': 'Ellensburg'}]},
 {'name': 'Kickstarter',
  'category_code': 'design',
  'offices': [{'city': 'Brooklyn'}]},
 {'name': 'Moonfruit',
  'category_code': 'design',
  'offices': [{'city': 'London'}]}]

In [12]:
#Checking that the distribution of companies of design and games_video is similar to all companies --> NY, SF and London as finalists
condition_2 = {"category_code": {"$in": ["design", "games_video"]}}
projection = {"_id":0, "name":1, "category_code":1, "offices.city":1}
category_list = list(c.find(condition_2, projection))

category_city = []
for company in category_list:
    for office in company["offices"]:
        category_city.append(office["city"])
        
count = pd.Series(category_city).value_counts()
print("Category Count")
print(count.head())

Category Count
New York         75
San Francisco    69
                 54
London           37
Los Angeles      31
Name: count, dtype: int64


In [13]:
#Adding conditions to get a finalists list
condition_category = {"category_code": {"$in": ["design", "games_video"]}}
condition_city = {"offices.city": "London"}
condition_raised_amount = {"funding_rounds.raised_amount": {"$gte": 1000000}}
projection = {"_id":0, "name":1, "category_code":1, "offices.city":1, "funding_rounds.raised_amount":1, "founded_year":1, "offices.longitude":1, "offices.latitude":1}
finalists = list(c.find({"$and": [condition_category, condition_city,condition_raised_amount]}, projection).sort("founded_year",-1))

In [14]:
#Df of final results that might be a good fit for the company offices 
data = []
for company in finalists:
    name = company["name"]
    category = company["category_code"]
    offices = company.get("offices", [])
    for office in offices:
        city = office.get("city", "")
        latitude = office.get("latitude", None)
        longitude = office.get("longitude", None)
        if city == "London" and latitude is not None and longitude is not None:
            data.append({"Company": name, "Latitude": latitude, "Longitude": longitude, "Category": category})

df = pd.DataFrame(data)
print(df.to_string(index=False))

     Company  Latitude  Longitude    Category
    Babelgum 53.344104  -6.267494 games_video
    Playfish 51.499109  -0.198480 games_video
       Pikum 51.539778  -0.152998 games_video
     WorldTV 51.500152  -0.126236 games_video
Wazoo Sports 37.128600 -84.083400 games_video


In [15]:
#Babelgum in Ireland
#WorldTV and Kublax same location
#Wazoo Sports in Pittsburg, USA 

#Choosen companies: 
    #Playfish
    #Pikum
    #WorldTV
    

In [16]:
#Creating a map of London city
london_lat = 51.509865
london_lon = -0.118092
london_map = folium.Map(location = [london_lat, london_lon], zoom_start=12)

In [17]:
#Adding the 3 companies markers
playfish_lat = 51.539778
playfish_long = -0.152998
playfish = Marker(location = [playfish_lat, playfish_long], tooltip="Playfish")
playfish.add_to(london_map)
worldtv_lat = 51.500152 
worldtv_long = -0.126236
worldtv = Marker(location = [worldtv_lat, worldtv_long], tooltip="WorldTV")
worldtv.add_to(london_map)
pikum_lat = 51.499109 
pikum_long = -0.198480
pikum = Marker(location = [pikum_lat, pikum_long], tooltip="Pikum")
pikum.add_to(london_map)
london_map

In [18]:
london_map

In [19]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map

In [20]:
#Companies are placed in Camden, Kensington and Westminster, we will check their sorroundings to decide.

In [21]:
# Requests to 4square

In [22]:
def requests_for_foursquare (query, lat, lon, radius=500, limit=50):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}&sort=DISTANCE"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [23]:
def extracting_from_one_element (one_venue):

    name = one_venue["name"]
    distance = one_venue["distance"]
    lat = one_venue["geocodes"]["main"]["latitude"]
    lon = one_venue["geocodes"]["main"]["longitude"]

    
    small_dict = {
    "name": name,
    "distance": distance,
    "location": [lon,lat]
}
    return small_dict

In [24]:
def transform_request_into_df (res):
    list_of_dictionaries = [extracting_from_one_element (element) for element in res["results"]]
    return pd.DataFrame(list_of_dictionaries)

In [25]:
def request_to_df (query):
    pikum = requests_for_foursquare (query, pikum_lat, pikum_long, radius=1000, limit=50)
    df_pikum = transform_request_into_df (pikum)

    worldtv = requests_for_foursquare (query, worldtv_lat, worldtv_long, radius=1000, limit=50)
    df_worldtv = transform_request_into_df (worldtv)

    playfish = requests_for_foursquare (query, playfish_lat, playfish_long, radius=1000, limit=50)
    df_playfish = transform_request_into_df (playfish)
    
    merged_df = pd.concat([df_pikum, df_worldtv, df_playfish]).reset_index(drop=True)
    
    return merged_df

In [26]:
def df_to_json (df,file_name):
    df.to_json(f'data/{file_name}.json', orient='records')

In [27]:
def import_mongo (file_name,collection_name):
    client = pymongo.MongoClient("localhost:27017")
    db = client["London"]
    c = db.get_collection(f'{collection_name}')
    venues = pd.read_json(f'data/{file_name}.json').to_dict(orient='records')
    c.insert_many(venues)

In [28]:
starbucks = request_to_df ("starbucks")
df_to_json (starbucks,"london_starbucks")
import_mongo ("london_starbucks","london_starbucks")

In [29]:
schools = request_to_df ("school")
df_to_json (schools,"london_schools")
import_mongo ("london_schools","london_schools")

In [30]:
party = request_to_df (("bar","club"))
df_to_json (party,"london_party")
import_mongo("london_party","london_party")

In [31]:
dog_saloons = request_to_df ("dog saloon")
df_to_json (dog_saloons,"london_dogsaloons")
import_mongo ("london_dogsaloons", "london_dogsaloons")

In [32]:
vegans = request_to_df ("vegan restaurant")
df_to_json (vegans,"london_vegans")
import_mongo ("london_vegans","london_vegans")

In [33]:
#Basketball stadium (10 km)

pikum = requests_for_foursquare ("basketball stadium", pikum_lat, pikum_long, radius=10000, limit=50)
df_pikum = transform_request_into_df (pikum)

worldtv = requests_for_foursquare ("basketball stadium", worldtv_lat, worldtv_long, radius=10000, limit=50)
df_worldtv = transform_request_into_df (worldtv)

playfish = requests_for_foursquare ("basketball stadium", playfish_lat, playfish_long, radius=10000, limit=50)
df_playfish = transform_request_into_df (playfish)

basketball_stadiums = pd.concat([df_pikum, df_worldtv, df_playfish]).reset_index(drop=True)

basketball_json = basketball_stadiums.to_json(f'data/basketball_stadium.json', orient='records')

c = db.get_collection(f'london_basketball')
venues = pd.read_json(f'data/basketball_stadium.json').to_dict(orient='records')
c.insert_many(venues)

InsertManyResult([ObjectId('65496ea07f5523c29e49c5d1'), ObjectId('65496ea07f5523c29e49c5d2'), ObjectId('65496ea07f5523c29e49c5d3'), ObjectId('65496ea07f5523c29e49c5d4')], acknowledged=True)

In [34]:
def venues_london_df ():
    
    #1. For every collection
        # 2. For every neighborhood
    
    new_list = []
    venues = ["london_party", "london_schools","london_starbucks", "london_dogsaloon", "london_vegans"]
    neighborhoods = ["Camden", "Wesminster", "Kensington and Chelsea"]
    
    # 1. 
    for venue in venues:
        venues_connection = db.get_collection(venue)
        # 2. Loop the neighs
        for n in neighborhoods:
            londonneigh = db.get_collection("londonneigh")
            neighborhood_polygon = londonneigh.find_one({"properties.name": n})
            
            if neighborhood_polygon:
                results = list(venues_connection.find({
                    "location": {
                        "$geoWithin": {
                            "$geometry": neighborhood_polygon["geometry"]
                        }
                    }
                }))
                

                if results:
                    for i in results:
                        dict_ = {
                            "Venue name": i["name"],
                            "Category": venue,
                            "Lat": i["location"][1],
                            "Lon": i["location"][0],
                            "Neighborhood": n,
                        }

                        new_list.append(dict_)

    # Create a DataFrame from the result
    df = pd.DataFrame(new_list)

    return df

In [35]:
df = venues_london_df()
df = df.drop_duplicates()
df

,Venue name,Category,Lat,Lon,Neighborhood
0,Hampstead Comedy Club,london_party,51.542516,-0.154923,Camden
1,Barfly Club,london_party,51.542185,-0.156224,Camden
2,Guy Gold Coffee Bar and Treatment Rooms,london_party,51.539884,-0.147673,Camden
3,Guy Gold Coffee Bar and Treatment Rooms,london_party,51.539822,-0.147672,Camden
4,Torquil's Bar,london_party,51.543241,-0.152099,Camden
...,...,...,...,...,...
291,Wagamama High Street Kensington,london_vegans,51.502461,-0.189050,Kensington and Chelsea
292,Zizzi,london_vegans,51.492749,-0.194207,Kensington and Chelsea
293,Mali Vegan Thai,london_vegans,51.492695,-0.193050,Kensington and Chelsea
294,Masala,london_vegans,51.492717,-0.192579,Kensington and Chelsea


In [36]:
category_count = df.groupby(["Category", "Neighborhood"]).size().reset_index(name="Count")

# Pivot the table to get a proper format
category_count_table = category_count.pivot_table(index="Category", columns="Neighborhood", values="Count", fill_value=0)
category_count_table = category_count_table.astype(int)

# Print the result
print(category_count_table)

Neighborhood      Camden  Kensington and Chelsea  Wesminster
Category                                                    
london_party          47                      35          32
london_schools        49                      44          46
london_starbucks       2                       1           2
london_vegans         15                      12          10


In [37]:
#VISUALIZATION

In [38]:
def build_map(df, map_):
    for _, row in df.iterrows():
        
        if row["Category"] == 'london_starbucks':
            icon = folium.Icon(
                color = "darkgreen",
                icon_color = "white",
                icon = "fa-coffee",
                prefix = "fa",
            )
            
        elif row["Category"] == 'london_party':
            icon = folium.Icon(
                color = "black",
                icon_color = "white",
                icon = "fa-beer",
                prefix = "fa",
            )
        elif row["Category"] == 'london_vegans':
            icon = folium.Icon(
                color = "lightred",
                icon_color = "white",
                icon = "fa-leaf",
                prefix = "fa",
            )
            
        elif row["Category"] == 'london_schools':
            icon = folium.Icon(
                color = "darkblue",
                icon_color = "white",
                icon = "fa-graduation-cap",
                prefix = "fa",
            )

        
        new_marker = folium.Marker(location = [row["Lat"], row["Lon"]], icon=icon)
        new_marker.add_to(map_)
    return map_

In [39]:
#colors: {'gray', 'lightblue', 'darkred', 'lightred', 'purple', 'lightgreen', 'lightgray', 'black', 'darkgreen', 'white', 'darkpurple',
#'darkblue', 'pink', 'cadetblue', 'orange', 'red', 'blue', 'green', 'beige'}
#Creating a map of London city
london_lat = 51.509865
london_lon = -0.118092
london_map = Map(location = [london_lat, london_lon], zoom_start=12)
map_marked = build_map(df,london_map)
folium.Circle([pikum_lat, pikum_long], radius=500, fill_color='blue').add_to(map_marked)
folium.Circle([worldtv_lat, worldtv_long], radius=500, fill_color='blue').add_to(map_marked)
folium.Circle([playfish_lat, playfish_long], radius=500, fill_color='blue').add_to(map_marked)
map_marked

In [ ]:
#folium radio

In [ ]:
#Working with geojson

In [ ]:
"""
new_list = []
for i in london:
    new_dict = {
        "name": i["properties"]["name"],
        "geo": {"type": "Polygon", 
                 "coordinates": i["geometry"]["coordinates"][0]
                }
    }
    new_list.append(new_dict)
new_list
"""

In [ ]:
#from pymongo import MongoClient

In [ ]:
#london is the downloaded geojson (https://github.com/natzar/European-Neighborhoods-Json-Coords/blob/master/london.json)
# london[0]
# for i in london:
#     i['geometry']['coordinates'][0].append(i['geometry']['coordinates'][0][0])

In [ ]:
# client = MongoClient("localhost:27017")
# db = client["Ironhack"]
# c = db.get_collection("londonneigh")

# for i in london:
#     c.insert_one(i)